# Welcome to your first assignment!

Instructions are below. Please give this a try, and look in the solutions folder if you get stuck (or feel free to ask me!)

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">Just before we get to the assignment --</h2>
            <span style="color:#f71;">I thought I'd take a second to point you at this page of useful resources for the course. This includes links to all the slides.<br/>
            <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">https://edwarddonner.com/2024/11/13/llm-engineering-resources/</a><br/>
            Please keep this bookmarked, and I'll continue to add more useful links there over time.
            </span>
        </td>
    </tr>
</table>

# HOMEWORK EXERCISE ASSIGNMENT

Upgrade the day 1 project to summarize a webpage to use an Open Source model running locally via Ollama rather than OpenAI

You'll be able to use this technique for all subsequent projects if you'd prefer not to use paid APIs.

**Benefits:**
1. No API charges - open-source
2. Data doesn't leave your box

**Disadvantages:**
1. Significantly less power than Frontier Model

## Recap on installation of Ollama

Simply visit [ollama.com](https://ollama.com) and install!

Once complete, the ollama server should already be running locally.  
If you visit:  
[http://localhost:11434/](http://localhost:11434/)

You should see the message `Ollama is running`.  

If not, bring up a new Terminal (Mac) or Powershell (Windows) and enter `ollama serve`  
And in another Terminal (Mac) or Powershell (Windows), enter `ollama pull llama3.2`  
Then try [http://localhost:11434/](http://localhost:11434/) again.

If Ollama is slow on your machine, try using `llama3.2:1b` as an alternative. Run `ollama pull llama3.2:1b` from a Terminal or Powershell, and change the code below from `MODEL = "llama3.2"` to `MODEL = "llama3.2:1b"`

In [1]:
# imports

import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [2]:
# Constants

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

In [3]:
# Create a messages list using the same format that we used for OpenAI

messages = [
    {"role": "user", "content": "Describe some of the business applications of Generative AI"}
]

In [4]:
payload = {
        "model": MODEL,
        "messages": messages,
        "stream": False
    }

In [5]:
response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
print(response.json()['message']['content'])

Generative AI has numerous business applications across various industries, including:

1. **Content Generation**: AI can generate high-quality content such as blog posts, social media posts, product descriptions, and more. This can help businesses save time and effort in creating content.
2. **Image and Video Generation**: Generative AI can create realistic images and videos that can be used for advertising, marketing, and other purposes.
3. **Chatbots and Virtual Assistants**: AI-powered chatbots can engage with customers, answer questions, and provide support, freeing up human customer service representatives to focus on more complex issues.
4. **Product Design and Development**: Generative AI can assist in designing and developing new products by generating 2D and 3D models, prototypes, and product simulations.
5. **Marketing and Advertising**: AI can analyze consumer behavior, preferences, and trends to create personalized marketing campaigns and advertisements that are more effec

# Introducing the ollama package

And now we'll do the same thing, but using the elegant ollama python package instead of a direct HTTP call.

Under the hood, it's making the same call as above to the ollama server running at localhost:11434

In [6]:
import ollama

response = ollama.chat(model=MODEL, messages=messages)
print(response['message']['content'])

Generative AI has numerous business applications across various industries. Here are some examples:

1. **Content Generation**: AI can generate high-quality content such as articles, blog posts, social media posts, and even entire books. This can be particularly useful for businesses with limited content creation resources.
2. **Image and Video Creation**: Generative AI can create realistic images and videos that can be used in advertising, marketing materials, or even entertainment. For example, a company like IKEA has used generative AI to create high-quality product images without the need for expensive photography equipment.
3. **Chatbots and Virtual Assistants**: AI-powered chatbots can help businesses provide customer support 24/7, answering frequently asked questions and route complex issues to human agents. Generative AI can also be used to generate responses that are more natural and engaging.
4. **Marketing Campaigns**: Generative AI can be used to create personalized marketi

# NOW the exercise for you

Take the code from day1 and incorporate it here, to build a website summarizer that uses Llama 3.2 running locally instead of OpenAI

In [7]:
# imports
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
import ollama


In [8]:
# Constants
MODEL = "llama3.2"

In [9]:
# A class to represent a Webpage

class Website:
    """
    A utility class to represent a Website that we have scraped
    """

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [10]:
# Define our system prompt

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [11]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [12]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [13]:
def summarize(url):
    website = Website(url)
    messages = messages_for(website)
    response = ollama.chat(model=MODEL, messages=messages)
    return response['message']['content']

In [14]:
summarize("https://edwarddonner.com")

"### Summary of Edward Donner's Website\n\n#### About the Author\nEdward Donner is a writer, entrepreneur, and expert in Large Language Models (LLMs). He enjoys DJing, music production, and reading Hacker News. As the co-founder and CTO of Nebula.io, he applies AI to help people discover their potential.\n\n#### Key Projects and Initiatives\n\n*   **Nebula.io**: Developing AI-powered talent management solutions using proprietary LLMs.\n*   **untapt**: Founded and previously led as CEO before its acquisition in 2021.\n\n#### News and Announcements\nThe website features recent news articles, including:\n\n*   [Mastering AI and LLM Engineering – Resources](November 13, 2024)\n*   [From Software Engineer to AI Data Scientist – resources](October 16, 2024)\n*   [Choosing the Right LLM: Toolkit and Resources](June 26, 2024)\n\n### Contact Information\nTo get in touch with Edward Donner, follow these links:\n\n*   Email: ed [at] edwarddonner [dot] com\n*   Website: www.edwarddonner.com\n*   L

In [15]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [16]:
display_summary("https://edwarddonner.com")

# Summary of Edward Donner's Website
=====================================

### Introduction

Edward Donner is the co-founder and CTO of Nebula.io, a company applying AI to help people discover their potential. He shares his interests in writing code, experimenting with LLMs, DJing, and amateur electronic music production.

### Recent Announcements
-------------------------

* **Mastering AI and LLM Engineering – Resources** (November 13, 2024): A collection of resources for improving AI and LLM engineering skills.
* **From Software Engineer to AI Data Scientist – resources** (October 16, 2024): Resources for transitioning from a software engineer role to an AI data scientist role.
* **Outsmart LLM Arena – a battle of diplomacy and deviousness** (June 26, 2024): A new arena where LLMs compete against each other in a game of diplomacy and strategy.
* **Choosing the Right LLM: Toolkit and Resources** (August 6, 2024): A toolkit and resources for choosing the right LLM for specific use cases.

### Other Content

The website features an "Outsmart" arena where LLMs compete against each other in a battle of diplomacy and deviousness. It also includes information about Edward Donner's background as a founder and CEO of AI startups, and his current work at Nebula.io.